# Merge probes for CTP-12 marker gene DNA RNA

There are 2 different sources:
1. TSS 150 probe library for cortex marker genes
2. corresponding smFISH from Meng's M1 library used in paper: https://www.nature.com/articles/s41586-021-03705-x

In [1]:
%run "..\..\Startup_py3.py"
sys.path.append(r"..\..\..\..\Documents")

import ImageAnalysis3
from ImageAnalysis3 import get_img_info, visual_tools, corrections, library_tools

from ImageAnalysis3.library_tools import LibraryDesigner as ld
from ImageAnalysis3.library_tools import LibraryTools

%matplotlib notebook
print(os.getpid())

# biopython for SeqRecord
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

# blast
from Bio.Blast.Applications import NcbiblastnCommandline
from Bio.Blast import NCBIXML 
import ImageAnalysis3.library_tools.quality_check as qc

14600


In [2]:
# Library directories
pool_folder = r'\\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-12_mop_markers'
sub_fds = [os.path.join(pool_folder, _fd) for _fd in os.listdir(pool_folder) if os.path.isdir(os.path.join(pool_folder, _fd))]


## scan probe files

In [3]:
files = []
library_names = []
for _fd in sub_fds:
    if 'blast_full_probes.fasta' in os.listdir(_fd):
        files.append(os.path.join(_fd, 'blast_full_probes.fasta'))
        library_names.append(os.path.basename(_fd).replace('_', '-'))
    elif 'filtered_full_probes.fasta' in os.listdir(_fd):
        files.append(os.path.join(_fd, 'filtered_full_probes.fasta'))
        library_names.append(os.path.basename(_fd).replace('_', '-'))
        
print(files)
print(library_names)

['\\\\10.245.74.212\\Chromatin_NAS_2\\Chromatin_Libraries\\CTP-12_mop_markers\\smFISH_cell_class\\filtered_full_probes.fasta', '\\\\10.245.74.212\\Chromatin_NAS_2\\Chromatin_Libraries\\CTP-12_mop_markers\\marker_gene_TSS\\blast_full_probes.fasta']
['smFISH-cell-class', 'marker-gene-TSS']


## load sub-libraries

In [4]:
target_len = 120
library_2_pbs = {}

for _fl, _lib_name in zip(files, library_names):
    print(f"loading probes from file: {_fl}")
    _records = []
    with open(_fl, 'r') as _handle:
        for _record in SeqIO.parse(_handle, "fasta"):
            # modify names
            _record.id = _record.id + f"_library_{_lib_name}"
            _record.name, _record.description = '',''
            # modify seq if length doesn't match
            if len(_record.seq) < target_len:
                _seq = _record.seq + Seq('A'*(target_len - len(_record.seq)))
                _record.seq = _seq
        
            _records.append(_record)
        
    library_2_pbs[_lib_name] = _records

loading probes from file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-12_mop_markers\smFISH_cell_class\filtered_full_probes.fasta
loading probes from file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-12_mop_markers\marker_gene_TSS\blast_full_probes.fasta


## merge and save

In [5]:
# merge into one file and save
final_pool_records = []
for _lib_name, _records in library_2_pbs.items():
    print(f"appending {len(_records)} probes from library: {_lib_name}")
    final_pool_records.extend(_records)

appending 2260 probes from library: smFISH-cell-class
appending 3360 probes from library: marker-gene-TSS


In [6]:
save_filename = os.path.join(pool_folder, 'merged_CTP12.fasta')
print(f"number of probes saved: {len(final_pool_records)}")
with open(save_filename, 'w') as _output_handle:
    SeqIO.write(final_pool_records, _output_handle, "fasta")

number of probes saved: 5620


# Calculate concentration

In [44]:
# RNA
vol = 9e3 # ul
conc = 145.7e-9 # g/ul
num_pb = 14709
pb_len = 150
pb_mw = (pb_len * 303.7 + 79) # g/mol

mass_total = vol * conc
mass_per_pb = mass_total / num_pb
mol_per_pb = mass_per_pb / pb_mw
print(mass_total)
print(mass_per_pb)
print(mol_per_pb)
target_conc = 1e-8 * 1e-6 # mol/ul
final_vol = mol_per_pb / target_conc
print("final in ul:", final_vol)

0.0013112999999999998
8.914950030593513e-08
1.953576287547336e-12
final in ul: 195.3576287547336


In [46]:
# DNA TSS
vol = 9e3 # ul
conc = 165.1e-9 # g/ul
num_pb = 54485
pb_len = 140
pb_mw = (pb_len * 303.7 + 79) # g/mol

mass_total = vol * conc
mass_per_pb = mass_total / num_pb
mol_per_pb = mass_per_pb / pb_mw
print(mass_total)
print(mass_per_pb)
print(mol_per_pb)
target_conc = 1e-8 * 1e-6 # mol/ul
final_vol = mol_per_pb / target_conc
print("final in ul:", final_vol)

0.0014858999999999999
2.727172616316417e-08
6.402264517023305e-13
final in ul: 64.02264517023305


In [47]:
# DNA TSS
vol = 9e3 # ul
conc = 169.8e-9 # g/ul
num_pb = 26147
pb_len = 140
pb_mw = (pb_len * 303.7 + 79) # g/mol

mass_total = vol * conc
mass_per_pb = mass_total / num_pb
mol_per_pb = mass_per_pb / pb_mw
print(mass_total)
print(mass_per_pb)
print(mol_per_pb)
target_conc = 1e-8 * 1e-6 # mol/ul
final_vol = mol_per_pb / target_conc
print("final in ul:", final_vol)

0.0015282
5.8446475695108425e-08
1.3720796228633102e-12
final in ul: 137.20796228633102


8.914950030593513e-08
